# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [24]:
from xml.etree import ElementTree as ET


## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [3]:
document = ET.parse("/Users/shashank/Desktop/SpringboardDatascience/data_wrangling_xml/data/mondial_database.xml")

In [45]:
root=document.getroot()
root

<Element 'mondial' at 0x104ec8908>

In [65]:
#Q1. 10 countries with the lowest infant mortality rates
#create a dictionary that will be used

data = {'country':[], 'infant_mortality':[]}
   
    
                       

In [72]:
 #get the name and infant mortality of all countries
for element in document.iterfind('country'):
    data['country'].append(element.find('name').text)
    mortality = element.find('infant_mortality')
    if mortality is not None:
        data['infant_mortality'].append(mortality.text)
    else:
        data['infant_mortality'].append('')

In [73]:

import pandas as pd
import numpy as np

In [74]:
 #create a pandas dataframe from the data
df = pd.DataFrame(data)

# clean up countries with missing infant_mortality rates
df = df.replace('',np.NaN)

# find the 10 lowest infant mortality rates
df.dropna().sort(columns='infant_mortality').head(10)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,country,infant_mortality
38,Monaco,1.81
30,Romania,10.16
153,Fiji,10.2
69,Brunei,10.48
132,Grenada,10.5
237,Mauritius,10.59
124,Panama,10.7
243,Seychelles,10.77
102,United Arab Emirates,10.92
113,Barbados,10.93


In [76]:
#10 cities with the largest population

In [114]:
data = {'city':[], 'country':[], 'year':[], 'population':[]}

# extract city name, population, and year from xml
for city in document.findall('country/city'):
    country = city.attrib['country']
    name = ''
    population = 0
    year = 0
    for node in list(city):
        if node.tag == 'name':
            name = node.text
        elif node.tag == 'population':
            # note: if multiple population tags, the last is used
            # population tags sorted by year, so last is most recent
            population = int(node.text)
            year = int(node.attrib['year'])
    
    # only add cities with known populations
    if population != 0:
        data['country'].append(country)
        data['city'].append(name)
        data['population'].append(population)
        data['year'].append(year)


In [116]:
df = pd.DataFrame(data)
df.sort(columns='population', ascending=False).head(10)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,city,country,population,year
165,Seoul,ROK,9708483,2010
154,Cairo,ET,8471859,2006
75,Bangkok,THA,7506700,1999
123,Hong Kong,HONX,7055071,2009
87,Ho Chi Minh City,VN,5968384,2009
201,Singapore,SGP,5076700,2010
153,Alexandria,ET,4123869,2006
205,New Taipei,RC,3939305,2012
166,Pusan,ROK,3403135,2010
102,Pyongyang,NOK,3255288,2008


In [98]:
#Q3-10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [119]:
data = {'group':[], 'country':[], 'population':[]}

# extract country, ethnic group, and population from xml
for country in document.findall('country'):
    name = ''
    group = ''
    population = 0
    for node in list(country):
        if node.tag == 'name':
            name = node.text
        elif node.tag == 'population':
            # note: if multiple population tags, the last is used
            # population tags sorted by year, so last is most recent
            population = int(node.text)
            year = int(node.attrib['year'])
        elif node.tag == 'ethnicgroup':
            percentage = float(node.attrib['percentage'])
            group = node.text
            
            if group and percentage and population:
            # only add groups with known populations
                data['country'].append(name)
                data['group'].append(group)
                # calculate ethnic group population from percentage of overall population
                data['population'].append(int(population * percentage / 100.))

In [120]:
# create a pandas dataframe from the data
df = pd.DataFrame(data)

total = df.groupby('group')[['population']].sum()

# find the 10 largest ethnic groups, by population
total.sort(columns='population', ascending=False).head(10)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,population
group,
Han Chinese,1245058800
Indo-Aryan,871815583
European,494872201
African,318325104
Dravidian,302713744
Mestizo,157734349
Bengali,146776916
Russian,131856989
Japanese,126534212


In [ ]:
#name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [122]:
# create a lookup table for country codes
car_codes = {'country':[], 'code':[]}

# extract country info from xml
for country in document.findall('country'):
    name = ''
    code = ''
    for node in list(country):
        if node.tag == 'name':           
            car_codes['country'].append(node.text)
            car_codes['code'].append(country.attrib['car_code'])

# create a dataframe for the country codes

countries = pd.DataFrame(car_codes)
countries.head(10)

,code,country
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro
5,KOS,Kosovo
6,AND,Andorra
7,F,France
8,E,Spain
9,A,Austria


In [123]:
# create a table for river information
river_data = {'car_code':[], 'name':[], 'length':[]}

# extract river info from xml
for river in document.findall('river'):
    
    code = river.attrib['country'] # country code, not country name
    name = ''
    length = 0
    for node in list(river):
        if node.tag == 'name':           
            name = node.text
        elif node.tag == 'length':
            length = float(node.text)
            
    if name and code and length:
        # only add rivers with known lengths
        codes = code.split(' ') # some rivers cross multiple countries
        for c in codes:
            river_data['car_code'].append(c)
            river_data['name'].append(name) # river name
            river_data['length'].append(length)

# create a dataframe for the rivers
rivers = pd.DataFrame(river_data)
rivers.head(10)

,car_code,length,name
0,IS,230.0,Thjorsa
1,IS,206.0,Joekulsa a Fjoellum
2,N,604.0,Glomma
3,N,322.0,Lagen
4,S,93.0,Goetaaelv
5,N,460.0,Klaraelv
6,S,460.0,Klaraelv
7,S,470.0,Umeaelv
8,S,520.0,Dalaelv
9,S,320.0,Vaesterdalaelv


In [124]:
# create a table for lake information
lake_data = {'car_code':[], 'name':[], 'area':[]}

# extract lake info from xml
for river in document.findall('lake'):
   
    code = river.attrib['country'] # country code, not country name
    name = ''
    area = 0
    for node in list(river):
        if node.tag == 'name':           
            name = node.text
        elif node.tag == 'area':
            area = float(node.text)
            
    if name and code and area:
        # only add lakes with known areas
        codes = code.split(' ') # some lakes cross multiple countries
        for c in codes:
            lake_data['car_code'].append(c)
            lake_data['name'].append(name) # lake name
            lake_data['area'].append(area)

# create a dataframe for the lakes
lakes = pd.DataFrame(lake_data)
lakes.head(10)

,area,car_code,name
0,1040.0,SF,Inari
1,928.0,SF,Oulujaervi
2,472.0,SF,Kallavesi
3,4370.0,SF,Saimaa
4,1118.0,SF,Paeijaenne
5,368.0,N,Mjoesa-See
6,173.0,S,Storuman
7,290.0,S,Siljan
8,1140.0,S,Maelaren
9,5648.0,S,Vaenern


In [125]:
# create a table for airport information
airport_data = {'car_code':[], 'name':[], 'elevation':[]}

# extract airport info from xml
for airport in document.findall('airport'):
   
    code = airport.attrib['country'] # country code, not country name
    name = ''
    elev = 0
    for node in list(airport):
        if node.tag == 'name':           
            name = node.text
        elif node.tag == 'elevation':
            if node.text is not None:
                elev = float(node.text)
            
    if name and code and elev:
        # only add lakes with known areas
        #codes = code.split(' ') # some lakes cross multiple countries
        #for c in codes:
        airport_data['car_code'].append(code)
        airport_data['name'].append(name) # airport name
        airport_data['elevation'].append(elev)

# create a dataframe for the lakes
airports = pd.DataFrame(airport_data)
airports.head(10)

,car_code,elevation,name
0,AFG,977.0,Herat
1,AFG,1792.0,Kabul Intl
2,AL,38.0,Tirana Rinas
3,DZ,811.0,Cheikh Larbi Tebessi
4,DZ,822.0,Batna Airport
5,DZ,6.0,Soummam
6,DZ,1377.0,Tamanrasset
7,DZ,88.0,Biskra
8,DZ,691.0,Mohamed Boudiaf Intl
9,DZ,1024.0,Ain Arnat Airport


In [135]:
# merge the river and country tables
df = pd.merge(rivers, countries, how='left', left_on='car_code', right_on='code')

# find the longest river
grouped = df.groupby(['name'])[['length']].max()
longest = grouped.sort('length', ascending=False).ix[0]

# display info on longest river
print('Longest River:')
df[df.name == longest.name][['name','country','length']]

Longest River:


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,name,country,length
298,Amazonas,Colombia,6448.0
299,Amazonas,Brazil,6448.0
300,Amazonas,Peru,6448.0


In [131]:
# merge the lake and country tables
df = pd.merge(lakes, countries, how='left', left_on='car_code', right_on='code')

# find the longest river
grouped = df.groupby(['name'])[['area']].max()
largest = grouped.sort('area', ascending=False).ix[0].name

# display info on longest river
print ('Largest Lake:\n')
df[df.name == largest][['name','country','area']]

Largest Lake:



//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,name,country,area
68,Caspian Sea,Russia,386400.0
69,Caspian Sea,Azerbaijan,386400.0
70,Caspian Sea,Kazakhstan,386400.0
71,Caspian Sea,Iran,386400.0
72,Caspian Sea,Turkmenistan,386400.0


In [133]:
# merge the airport and country tables
df = pd.merge(airports, countries, how='left', left_on='car_code', right_on='code')

# find the highest airport
grouped = df.groupby(['name'])[['elevation']].max()
highest = grouped.sort('elevation', ascending=False).iloc[0].name

# display info on highest airport
airport = df[df.name == highest].iloc[0]
airport

print ('Highest Airport:')
print( '%s in %s, Elevation: %d m' % (airport['name'], airport.country, airport.elevation))

Highest Airport:
El Alto Intl in Bolivia, Elevation: 4063 m


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
